# SUPERVISED LEARNING TASK

In [ ]:
import numpy as np
import json
import glob
import sys
import os
from natsort import natsorted

from sklearn.linear_model import LinearRegression
from sklearn import svm

## WITHOUT DYNAMIC INHIBITION

In [ ]:
name_net = "..." # network name
name_type = "cSTDP2"
name_file_inh = "..." # inhibitory file
name_file = "..." # no inhibition file

name_learnt = "gestures"
name_random = "gestures_rd"
name_feedforward = "gesturesno"

In [ ]:
dataset_dir = "/.../"+name_net+ "/statistics/"+name_file+"/"

In [ ]:
training_dir = "/.../"+name_net+ "/statistics/"+name_file+"train/"
testing_dir = "/.../"+name_net+ "/statistics/"+name_file+"test/"

training_dir_x = training_dir + "x/"
training_dir_y = training_dir + "y/"

testing_dir_x = testing_dir + "x/"
testing_dir_y = testing_dir + "y/"

In [ ]:
training_dir_inh = "/.../"+name_net+ "/statistics/"+name_file_inh+"train/"
testing_dir_inh = "/.../"+name_net+ "/statistics/"+name_file_inh+"test/"

training_dir_xinh = training_dir_inh + "x/"
training_dir_yinh = training_dir_inh + "y/"

testing_dir_xinh = testing_dir_inh + "x/"
testing_dir_yinh = testing_dir_inh + "y/"

In [ ]:
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
os.mkdir(training_dir)
os.mkdir(testing_dir)

os.mkdir(training_dir_x)
os.mkdir(training_dir_y)

os.mkdir(testing_dir_x)
os.mkdir(testing_dir_y)

In [ ]:
it_train = 0
it_test = 0
rng = np.random.default_rng()

for label in labels:
    folder = dataset_dir + "/" + str(label) + "/"
    files = natsorted(glob.glob(os.path.join(folder, '*.json')))
    n_files = len(files)
    thresh = round(80 * n_files / 100) 
    v = True
    for ct, l in enumerate(range(n_files)):
        with open(files[l]) as file:
            params = json.load(file)
        x = params["x"]
        y = params["y"]
        
        if(ct+1 <= thresh):
            np.save(training_dir_x + str(it_train) + ".npy", x)
            np.save(training_dir_y + str(it_train) + ".npy", y)
            it_train+=1
        else:
            np.save(testing_dir_x + str(it_test) + ".npy", x)
            np.save(testing_dir_y + str(it_test) + ".npy", y)
            it_test+=1

In [ ]:
train = True

In [ ]:
X_train = []
Y_train = []

X_test = []
Y_test = []

X_train_inh = []
Y_train_inh = []

X_test_inh = []
Y_test_inh = []

In [ ]:
if(train):
    files_train_x = natsorted(glob.glob(os.path.join(training_dir_x, '*.npy')))
    files_train_y = natsorted(glob.glob(os.path.join(training_dir_y, '*.npy')))

    files_train_xinh = natsorted(glob.glob(os.path.join(training_dir_xinh, '*.npy')))
    files_train_yinh = natsorted(glob.glob(os.path.join(training_dir_yinh, '*.npy')))

else:
    files_test_x = natsorted(glob.glob(os.path.join(testing_dir_x, '*.npy')))
    files_test_y = natsorted(glob.glob(os.path.join(testing_dir_y, '*.npy')))

    files_test_xinh = natsorted(glob.glob(os.path.join(testing_dir_xinh, '*.npy')))
    files_test_yinh = natsorted(glob.glob(os.path.join(testing_dir_yinh, '*.npy')))

In [ ]:
if(train):
    print(len(files_train_y))

In [ ]:
if(not train):
    print(len(files_test_x))

In [ ]:
all_kernels = True
which = 0

In [ ]:
n_simple = 5184
n_complex = 1152
def CutKernel(a, desc):
    if (a == 0): # all
        X_simple = []
        X_complex = []
        for e in range(0, len(desc), n_simple + n_complex):
            X_simple.append(desc[e: e + n_simple])
            X_complex.append(desc[e + n_simple: e + n_simple + n_complex])
        X_simple = np.array(X_simple).flatten()
        X_complex = np.array(X_complex).flatten()
        return X_simple, X_complex
    X_ = []
    for e in range(0, len(desc), n_simple + n_complex):
        if(a == 1): # simple
            X_.append(desc[e: e + n_simple])
        if(a == 2): # complex
            X_.append(desc[e + n_simple: e + n_simple + n_complex])
    X_ = np.array(X_).flatten()
    return X_

In [ ]:
train_nz = []
test_nz = []
skip = 1
lim_files_training = 40000 
lim_files_val = 2000 
lim_files_testing = 50000 
ct_zero = []

if(train):
    for i in range(0,len(files_train_x)):
        if(i%skip==0 and len(X_train)<lim_files_training):
            if(np.max(np.load(files_train_x[i],allow_pickle=True))!=0):
                if(not all_kernels):
                    X_ = np.load(files_train_x[i],allow_pickle=True).flatten();
                    X__ = []
                    for e in range(0,len(X_),5184):
                        X__.append(np.reshape(X_[e:e+5184], (81,64)))
                    X_train.append(np.mean(np.array(X__),axis=1).flatten() / np.max(np.mean(np.array(X__),axis=1).flatten()))
                else:
                    sa = np.load(files_train_y[i]).flatten()
                    var = np.load(files_train_x[i],allow_pickle=True).flatten() 
                    var_inh = np.load(files_train_xinh[i],allow_pickle=True).flatten() 
                    if(which !=0):
                        var = CutKernel(which, var)
                        var_inh = CutKernel(which, var_inh)
                        if(np.max(var)!=0):
                            X_train.append(var)
                            size_init = int(np.sum(X_train[-1]))
                            size_inhib = int(np.sum(var_inh) / divide)
                            for value in range(size_init - size_inhib):
                                replace = np.where(X_train[-1]!=0)[0]
                                length = len(replace)
                                v = np.random.randint(length)
                                X_train[-1][replace[v]] = X_train[-1][replace[v]] - 1
                            train_nz.append(np.sum(X_train[-1]))
                    else:
                        var_simple, var_complex = CutKernel(which, var)
                        var_inh_simple, var_inh_complex = CutKernel(which, var_inh)
                        if(np.max(var_simple)!=0):
                            size_init = int(np.sum(var_simple))
                            size_inhib = int(np.sum(var_inh_simple) / divide)
                            for value in range(size_init - size_inhib):
                                replace = np.where(var_simple!=0)[0]
                                length = len(replace)
                                v = np.random.randint(length)
                                var_simple[replace[v]] = var_simple[replace[v]] - 1
                        if(np.max(var_complex)!=0):
                            size_init = int(np.sum(var_complex))
                            size_inhib = int(np.sum(var_inh_complex) / divide)
                            for value in range(size_init - size_inhib):
                                replace = np.where(var_complex!=0)[0]
                                length = len(replace)
                                v = np.random.randint(length)
                                var_complex[replace[v]] = var_complex[replace[v]] - 1
                        X_train.append(np.concatenate( (var_simple, var_complex), axis = 0))
                        train_nz.append(np.sum(X_train[-1])) 
            else:
                print("File number {} skipped because it was equal to 0.".format(i+1))
                continue
            if(np.max(var!=0)):
                Y_train.append(np.load(files_train_y[i]).flatten())

                print("File number (train) {0}/{1}".format(i+1, len(files_train_x)))

else:
    for i in range(0, len(files_test_x)):    
        if(i<len(files_test_x) and len(X_test)<lim_files_testing):
                if(np.max(np.load(files_test_x[i]))!=0):
                    if(not all_kernels):
                        X_ = np.load(files_test_x[i],allow_pickle=True).flatten();
                        X__ = []
                        for e in range(0,len(X_),5184):
                            X__.append(np.reshape(X_[e:e+5184], (81,64)))
                        X_test.append(np.mean(np.array(X__),axis=1).flatten() / np.max(np.mean(np.array(X__),axis=1).flatten()))
                    else:
                        sa = np.load(files_test_y[i]).flatten()
                        var = np.load(files_test_x[i],allow_pickle=True).flatten() 
                        var_inh = np.load(files_test_xinh[i],allow_pickle=True).flatten() 
                        if(which!=0):
                            var = CutKernel(which, var)
                            var_inh = CutKernel(which, var_inh)
                            if(np.max(var)!=0):
                                X_test.append(var)
                                size_init = int(np.sum(X_test[-1]) )
                                size_inhib = int(np.sum(var_inh) / divide)
                                init_replace = np.where(X_test[-1]!=0)[0]
                                for value in range(size_init - size_inhib):
                                    replace = np.where(X_test[-1]!=0)[0]
                                    length = len(replace)
                                    v = np.random.randint(length)
                                    X_test[-1][replace[v]] = X_test[-1][replace[v]] - 1
                                test_nz.append(np.sum(X_test[-1]))
                        else:
                            var_simple, var_complex = CutKernel(which, var)
                            var_inh_simple, var_inh_complex = CutKernel(which, var_inh)
                            if(np.max(var_simple)!=0):
                                size_init = int(np.sum(var_simple))
                                size_inhib = int(np.sum(var_inh_simple) / divide)
                                for value in range(size_init - size_inhib):
                                    replace = np.where(var_simple!=0)[0]
                                    length = len(replace)
                                    v = np.random.randint(length)
                                    var_simple[replace[v]] = var_simple[replace[v]] - 1
                            if(np.max(var_complex)!=0):
                                size_init = int(np.sum(var_complex))
                                size_inhib = int(np.sum(var_inh_complex) / divide)
                                for value in range(size_init - size_inhib):
                                    replace = np.where(var_complex!=0)[0]
                                    length = len(replace)
                                    v = np.random.randint(length)
                                    var_complex[replace[v]] = var_complex[replace[v]] - 1
                            X_test.append(np.concatenate( (var_simple, var_complex), axis = 0))
                            test_nz.append(np.sum(X_test[-1])) 

                else:
                    print("File number {} skipped because it was equal to 0.".format(i+1))
                    continue
                if(np.max(var!=0)):
                    Y_test.append(np.load(files_test_y[i]).flatten())
                    print("File number (test) {0}/{1}".format(i+1, len(files_test_x)))

In [ ]:
if(train):
    print("min number = {}, median number = {}, max number = {}".format(np.min(train_nz), np.median(train_nz), np.max(train_nz)))

In [ ]:
if(not train):
    print("min number = {}, median number = {}, max number = {}".format(np.min(test_nz), np.median(test_nz), np.max(test_nz)))

In [ ]:
if(train):
    X_train = np.array(X_train) #/np.max(np.array(X_train))
    Y_train = np.array(Y_train) 
else:
    X_test = np.array(X_test) #/np.max(np.array(X_test))
    Y_test = np.array(Y_test)

In [ ]:
# len(X_test[0])

In [ ]:
# np.count_nonzero(X_train[0])

### WITHOUT NEURAL NETWORKS

In [ ]:
if(not train):
    import joblib
    path_home = ".../"+name_net+"/" + name_type + "/statistics/"
    classif_path = path_home + name_file_inh + "... .sav"

In [ ]:
if(not train):
    classif = joblib.load(classif_path)

In [ ]:
if(train):
    classif = svm.LinearSVC(max_iter=10000, C=1).fit(X_train, np.ravel(Y_train))

In [ ]:
if(train):
    counter=0
    good_number=0
    count = np.zeros((len(labels)))
    tot_number = np.zeros((len(labels)))
    v = [0, 1, 2]
    for feature in X_train:
        pred=classif.predict([feature]) 
        number = Y_train[counter]
        if pred.item()==number:
            good_number+=1
            count[np.where(np.array(labels)==number)[0][0]]+=1
        counter+=1
        tot_number[np.where(np.array(labels)==number)[0][0]]+=1
    print("The percentage of correct classification is: {}%.".format(good_number*100/counter))
    for num, val in enumerate(labels):
        print("The percentage of correct classification for number {} is: {}%".format(val, count[num]*100/tot_number[num]))


In [ ]:
if(train):
    import joblib
    path_home = "/.../"+name_net+ "/statistics/"
    classif_path = path_home + name_file_inh + "learnt.sav"

    joblib.dump(classif, classif_path)

In [ ]:
if(not train):
    counter=0
    good_number=0
    count = np.zeros((len(labels)))
    tot_number = np.zeros((len(labels)))
    y_true = []
    y_pred = []
    for feature in X_test:
        pred=classif.predict([feature]) 
        number = Y_test[counter]
        if pred.item()==number:
            good_number+=1
            count[np.where(np.array(labels)==number)[0][0]]+=1
        counter+=1
        tot_number[np.where(np.array(labels)==number)[0][0]]+=1
        y_true.append(number)
        y_pred.append(pred.item())
    for num, val in enumerate(labels):
        print("The percentage of correct classification for number {} is: {}%".format(val, count[num]*100/tot_number[num]))

    print()
    print("The percentage of correct classification is: {}%.".format(good_number*100/np.sum(tot_number)))
